# 기반 준비

## 모듈 임포팅

In [28]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import optimizers
from tensorflow.keras.layers import Dense

## 데이터 다운로드

In [29]:
!wget https://github.com/dhrim/keras_howto_2021/raw/master/common/iris.csv

--2021-07-25 10:41:39--  https://github.com/dhrim/keras_howto_2021/raw/master/common/iris.csv
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/dhrim/keras_howto_2021/master/common/iris.csv [following]
--2021-07-25 10:41:39--  https://raw.githubusercontent.com/dhrim/keras_howto_2021/master/common/iris.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2218 (2.2K) [text/plain]
Saving to: ‘iris.csv.1’

iris.csv.1          100%[===================>]   2.17K  --.-KB/s    in 0s      

2021-07-25 10:41:40 (34.1 MB/s) - ‘iris.csv.1’ saved [2218/2218]



In [30]:
!head iris.csv

septal_length,septal_width,petal_length,petal_width,class
6.4,2.8,5.6,2.2,2
5.0,2.3,3.3,1.0,1
4.9,2.5,4.5,1.7,2
4.9,3.1,1.5,0.1,0
5.7,3.8,1.7,0.3,0
4.4,3.2,1.3,0.2,0
5.4,3.4,1.5,0.4,0
6.9,3.1,5.1,2.3,2
6.7,3.1,4.4,1.4,1


## 데이터 로딩

In [31]:
iris = pd.read_csv("iris.csv")
iris.head()

,septal_length,septal_width,petal_length,petal_width,class
0,6.4,2.8,5.6,2.2,2
1,5.0,2.3,3.3,1.0,1
2,4.9,2.5,4.5,1.7,2
3,4.9,3.1,1.5,0.1,0
4,5.7,3.8,1.7,0.3,0


In [32]:
data = iris.to_numpy()
print(data.shape)
print(data[:5])

(120, 5)
[[6.4 2.8 5.6 2.2 2. ]
 [5.  2.3 3.3 1.  1. ]
 [4.9 2.5 4.5 1.7 2. ]
 [4.9 3.1 1.5 0.1 0. ]
 [5.7 3.8 1.7 0.3 0. ]]


## x, y 분리 전에 섞기

In [33]:
np.random.shuffle(data)
print(data[:5])

[[5.8 2.7 5.1 1.9 2. ]
 [7.7 2.6 6.9 2.3 2. ]
 [6.8 3.  5.5 2.  1. ]
 [5.7 2.8 4.1 1.3 1. ]
 [6.8 3.2 5.9 2.3 2. ]]


## x, y로 분리

In [6]:
x = data[:,:4]
y = data[:,4:]

In [7]:
print(x.shape)
print(y.shape)

(120, 4)
(120, 1)


In [8]:
print(x[:5])
print(y[:5])

[[6.4 2.8 5.6 2.2]
 [5.  2.3 3.3 1. ]
 [4.9 2.5 4.5 1.7]
 [4.9 3.1 1.5 0.1]
 [5.7 3.8 1.7 0.3]]
[[2.]
 [1.]
 [2.]
 [0.]
 [0.]]


## x, y 쌍을 섞기

In [37]:
indexes = np.arange(x.shape[0])
np.random.shuffle(indexes)

x = x[indexes]
y = y[indexes]

[0 1 2 3 4 5 6 7 8 9]
[118  68  13   6  77  71   1 110  41  23]


# 데이터 분리

## 수작업으로 데이터 분리

train:valid:test를 8:1:1로 쪼갬

우선 8:2로 먼저 쪼개고, 나머지 2를 1:1:로 쪼갬

In [9]:
# train과 remain로 쪼개기
split_index = int(len(x)*0.8)

train_x, remain_x = x[:split_index], x[split_index:]
train_y, remain_y = y[:split_index], y[split_index:]


# reamin을 validation과 test로 쪼개기
split_index = int(len(remain_x)*0.5)
print(split_index)

valid_x, test_x = remain_x[:split_index], remain_x[split_index:]
valid_y, test_y = remain_y[:split_index], remain_y[split_index:]

12


In [10]:
print(train_x.shape)
print(valid_x.shape)
print(test_x.shape)

(96, 4)
(12, 4)
(12, 4)


## scikit의 train_test_split()으로 분리

In [11]:
from sklearn.model_selection import train_test_split

train_x, remain_x, train_y, remain_y = train_test_split(x, y, test_size=0.2, shuffle=True)

valid_x, test_x, valid_y, test_y = train_test_split(remain_x, remain_y, test_size=0.5, shuffle=True)


In [12]:
print(train_x.shape)
print(valid_x.shape)
print(test_x.shape)

(96, 4)
(12, 4)
(12, 4)


# validation 데이터 적용

## fit()의 validation_data 사용

In [13]:
from sklearn.model_selection import train_test_split

train_x, remain_x, train_y, remain_y = train_test_split(x, y, test_size=0.2, shuffle=True)

valid_x, test_x, valid_y, test_y = train_test_split(remain_x, remain_y, test_size=0.5, shuffle=True)


In [14]:
model = keras.Sequential([
    keras.layers.Dense(10, activation='relu', input_shape=(4,)),
    keras.layers.Dense(10, activation='relu'),
    keras.layers.Dense(3, activation='softmax')
])

In [15]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [16]:
model.fit(train_x, train_y, epochs=100, validation_data=(valid_x, valid_y) )

Epoch 1/100
3/3 [==============================] - 3s 74ms/step - loss: 1.0485 - accuracy: 0.4583 - val_loss: 1.0031 - val_accuracy: 0.5000
Epoch 2/100
3/3 [==============================] - 0s 12ms/step - loss: 1.0373 - accuracy: 0.5208 - val_loss: 0.9941 - val_accuracy: 0.5000
Epoch 3/100
3/3 [==============================] - 0s 11ms/step - loss: 1.0272 - accuracy: 0.5729 - val_loss: 0.9850 - val_accuracy: 0.5000
Epoch 4/100
3/3 [==============================] - 0s 10ms/step - loss: 1.0159 - accuracy: 0.5938 - val_loss: 0.9751 - val_accuracy: 0.5000
Epoch 5/100
3/3 [==============================] - 0s 10ms/step - loss: 1.0053 - accuracy: 0.6458 - val_loss: 0.9639 - val_accuracy: 0.6667
Epoch 6/100
3/3 [==============================] - 0s 9ms/step - loss: 0.9936 - accuracy: 0.6771 - val_loss: 0.9510 - val_accuracy: 0.6667
Epoch 7/100
3/3 [==============================] - 0s 10ms/step - loss: 0.9801 - accuracy: 0.6771 - val_loss: 0.9366 - val_accuracy: 0.6667
Epoch 8/100
3/3 [====

In [17]:
loss, acc = model.evaluate(test_x, test_y)

print("loss :", loss)
print("acc :", acc)


1/1 [==============================] - 0s 14ms/step - loss: 0.3897 - accuracy: 1.0000
loss : 0.3896573781967163
acc : 1.0


## fit()의 validation_split 사용


In [23]:
from sklearn.model_selection import train_test_split

train_x, test_x, train_y, test_y = train_test_split(x, y, test_size=0.1, shuffle=True)


In [24]:
model = keras.Sequential([
    keras.layers.Dense(10, activation='relu', input_shape=(4,)),
    keras.layers.Dense(10, activation='relu'),
    keras.layers.Dense(3, activation='softmax')
])

In [25]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [26]:
model.fit(train_x, train_y, epochs=100, validation_split=0.2)

Epoch 1/100
3/3 [==============================] - 0s 63ms/step - loss: 2.4790 - accuracy: 0.0000e+00 - val_loss: 2.2041 - val_accuracy: 0.0000e+00
Epoch 2/100
3/3 [==============================] - 0s 12ms/step - loss: 2.3824 - accuracy: 0.0000e+00 - val_loss: 2.1222 - val_accuracy: 0.0000e+00
Epoch 3/100
3/3 [==============================] - 0s 10ms/step - loss: 2.2936 - accuracy: 0.0000e+00 - val_loss: 2.0452 - val_accuracy: 0.0000e+00
Epoch 4/100
3/3 [==============================] - 0s 10ms/step - loss: 2.2006 - accuracy: 0.0000e+00 - val_loss: 1.9714 - val_accuracy: 0.0000e+00
Epoch 5/100
3/3 [==============================] - 0s 9ms/step - loss: 2.1272 - accuracy: 0.0000e+00 - val_loss: 1.8979 - val_accuracy: 0.0000e+00
Epoch 6/100
3/3 [==============================] - 0s 10ms/step - loss: 2.0416 - accuracy: 0.0000e+00 - val_loss: 1.8277 - val_accuracy: 0.0000e+00
Epoch 7/100
3/3 [==============================] - 0s 12ms/step - loss: 1.9654 - accuracy: 0.0465 - val_loss: 1.7

In [27]:
loss, acc = model.evaluate(test_x, test_y)

print("loss :", loss)
print("acc :", acc)


1/1 [==============================] - 0s 17ms/step - loss: 0.5014 - accuracy: 0.9167
loss : 0.5014492869377136
acc : 0.9166666865348816
